In [ ]:
#Factors to consider
#Season Wise crops
#State Wise + District wise
#Water Availibility 
#Soil Condition
#Climate
#Market Condition
#Cost to cultivate and produce

    # x = dataset[['area','production','yield','rainfall','n','p','oc','k']]


In [ ]:
import pandas as pd
import numpy as np
summer = pd.read_csv('Report - All Crops Summer Season_output.csv')
kharif =  pd.read_csv('Report - Kharif_output.csv')
rabi =  pd.read_csv('Report - Rabi_output.csv')
nutrition = pd.read_csv('nutrition.csv')
nutrition.drop_duplicates(subset=None, keep='first', inplace=True)
current_dataset = 'summer'

In [ ]:
#basic
def yield_reccomendation(area,season,state,district=None):
    if season == "summer":
        dataset = summer
    elif season == "rabi":
        dataset = rabi
    else:
        dataset = kharif
    dataset = dataset[dataset['state'] == state]
    if district != None:
        dataset = dataset[dataset['district'] == district]
    dataset = dataset.sort_values(by=['year'],ascending=False)
    output = []
    crops_covered = []
    for index,row in dataset.iterrows():
        if row['crop'] not in crops_covered and row['yield'] > 0:
            output.append([row['crop'],row['year'],row['yield'],row['yield'] * area])
            crops_covered.append(row['crop'])
    processed_dataset = pd.DataFrame(np.array(output),columns=['crop','year','yield','production'])
    processed_dataset = processed_dataset.sort_values(by=['production'],ascending=False)
    return processed_dataset
yield_reccomendation(100,'summer','Maharashtra','PUNE')

,crop,year,yield,production
0,Other Cereals,2018-19,0.82,82.0
3,Sunflower,2013-14,0.63,63.0
1,Groundnut,2018-19,1.8,180.0
2,Maize,2018-19,1.1,110.00000000000001


In [ ]:
#ml
kharif_dates = ['JUN','OCT']
rabi_dates = ['NOV','APR']
summer_dates = ['MAR','JUN']

def get_processed_data():
    dataset = pd.read_csv('rainfall in india 1901-2015.csv')
    output = []
    for index,row in dataset.iterrows():
        kharif_rain = (row['JUN']+row['JUL']+row['AUG']+row['SEP']+row['OCT'])/5
        rabi_rain = (row['NOV']+row['DEC']+row['JAN']+row['FEB']+row['MAR']+row['APR'])/6
        summer_rain = (row['MAR']+row['APR']+row['MAY']+row['JUN'])/4
        state = row['SUBDIVISION']
        output.append([state,row['YEAR'],kharif_rain,rabi_rain,summer_rain])

    processed_dataset = pd.DataFrame(np.array(output),columns=['state','year','kharif','rabi','summer'])
    for col in ['kharif','rabi','summer']:
        processed_dataset[col] = processed_dataset[col].astype('float')
    processed_dataset['year'] = processed_dataset['year'].astype('int')
    processed_dataset['state'] = processed_dataset['state'].astype('string')
    return processed_dataset

def get_rainfall(rainfall_dataset,year,state,season):
    if year > 2015:
        year = 2015
    rainfall_dataset = rainfall_dataset[rainfall_dataset['year'] == year]
    return float(rainfall_dataset[rainfall_dataset['state'].str.contains("Andhra".upper())][season])


rainfall_dataset = get_processed_data()
get_rainfall(rainfall_dataset,1999,'Andhra Pradesh','kharif')

141.72

In [ ]:
nutrition['District'] = [str(row['District'].upper()) for index,row in nutrition.iterrows()]
nut_dist = set(list(nutrition['District']))
print(len(nut_dist))

649


In [ ]:
summer_dist = set(list(summer['district']))
print(len(summer_dist))

383


In [ ]:
%xmode plain
def add_rainfall_datapoints(seasonal_dataset,season):
    dataset = seasonal_dataset
    output = []
    for index,row in dataset.iterrows():
        state = row['state']
        year = int(row['year'].split('-')[0])
        rainwater = get_rainfall(rainfall_dataset,year,state,season)
        output.append(rainwater)
    dataset['rainfall'] = output
    return dataset

if current_dataset == "kharif":
    dataset = add_rainfall_datapoints(kharif,current_dataset)
elif current_dataset == "rabi":
    dataset = add_rainfall_datapoints(rabi,current_dataset)
else:
    dataset = add_rainfall_datapoints(summer,current_dataset)


Exception reporting mode: Plain


In [ ]:
def add_element_datapoints(dataset):
    n = []
    oc = []
    p = []
    k = []
    state_average = {}
    for index,row in nutrition.iterrows():
        state = row['State']
        average = nutrition[nutrition['State'] == state].describe().loc['mean']
        state_average[state.lower()] = average

    for index,row in dataset.iterrows():
        state = row['state']
        district = row['district']
        filtered_by_state = nutrition[nutrition['State']==state]
        flag = 0
        if len(filtered_by_state) > 0 :
            filtered_by_district = filtered_by_state[filtered_by_state['District']== district]
            if len(filtered_by_district) > 0:
                n.append(float(filtered_by_district['N']))
                oc.append(float(filtered_by_district['OC']))
                p.append(float(filtered_by_district['P']))
                k.append(float(filtered_by_district['K']))
                flag = 1
        if flag == 0:
            state = state.lower().strip()
            if state in state_average.keys():
                    
                n.append(state_average[state]['N'])
                oc.append(state_average[state]['OC'])
                p.append(state_average[state]['P'])
                k.append(state_average[state]['K'])
            else:
                n.append(None)
                oc.append(None)
                p.append(None)
                k.append(None)            
    dataset['n'] = n
    dataset['p'] = p
    dataset['oc'] = oc
    dataset['k'] = k

    #dataset = pd.concat([seasonal_dataset, nutrition], axis = 1,ignore_index=True, sort=False)
    return dataset
dataset = add_element_datapoints(dataset)


In [ ]:
dataset.isna().sum() 

crop           0
state          0
district       0
year           0
season         0
area           0
production    44
yield          0
rainfall       0
n              0
p              0
oc             0
k              0
dtype: int64

In [ ]:
dataset.head()

,crop,state,district,year,season,area,production,yield,rainfall,n,p,oc,k
0,Arecanut,Andaman and Nicobar Islands,,2009-10,Summer,4153.0,2080.0,0.50,34.425,99.2,89.44,75.286667,87.60
1,Arhar/Tur,Andaman and Nicobar Islands,,2009-10,Summer,331.0,164.0,0.50,34.425,99.2,89.44,75.286667,87.60
2,Bajra,Gujarat,KHEDA,2007-08,Summer,22800.0,55300.0,2.43,80.425,100.0,16.67,45.190000,16.35
3,Bajra,Gujarat,KHEDA,2006-07,Summer,14400.0,35800.0,2.49,72.075,100.0,16.67,45.190000,16.35
4,Bajra,Gujarat,KHEDA,2005-06,Summer,14100.0,30500.0,2.16,46.525,100.0,16.67,45.190000,16.35


In [ ]:
set(list(dataset['state']))

{'Andaman and Nicobar Islands',
 'Assam',
 'Bihar',
 'Gujarat',
 'Karnataka',
 'Kerala',
 'Madhya Pradesh',
 'Maharashtra',
 'Manipur',
 'Meghalaya',
 'Odisha',
 'Puducherry',
 'Tripura',
 'Uttar Pradesh',
 'Uttarakhand',
 'West Bengal'}

In [ ]:
set(list(nutrition['State']))

{'Andaman And Nicobar Islands',
 'Andhra Pradesh',
 'Arunachal Pradesh',
 'Assam',
 'Bihar',
 'Chhattisgarh',
 'Delhi',
 'Goa',
 'Gujarat',
 'Haryana',
 'Himachal Pradesh',
 'Jammu And Kashmir',
 'Jharkhand',
 'Karnataka',
 'Kerala',
 'Ladakh',
 'Madhya Pradesh',
 'Maharashtra',
 'Manipur',
 'Meghalaya',
 'Mizoram',
 'Nagaland',
 'Odisha',
 'Puducherry',
 'Punjab',
 'Rajasthan',
 'Sikkim',
 'Tamil Nadu',
 'Telangana',
 'The Dadra And Nagar Haveli And Daman And Diu',
 'Tripura',
 'Uttar Pradesh',
 'Uttarakhand',
 'West Bengal'}

In [ ]:
dataset.to_csv("processed_data.csv")

In [ ]:
import pandas as pd
from sklearn import preprocessing
import pickle
def normalize_dataset(dataset):
    # x = dataset[['area','production','yield','rainfall','n','p','oc','k']]
    x = dataset[['production','rainfall','n','p','oc','k']]

    x = x.values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    normalized_dataset = pd.DataFrame(x_scaled,columns = ['production','rainfall','n','p','oc','k'])
    #normalized_dataset = pd.DataFrame(x_scaled,columns = ['area','production','yield','rainfall','n','p','oc','k'])

    for col in dataset.iteritems():
        if col[0] in ['production','rainfall','n','p','oc','k']:
        #if col[0] in ['area','production','yield','rainfall','n','p','oc','k']:
            dataset[col[0]] = normalized_dataset[col[0]]
    le = preprocessing.LabelEncoder()
    le.fit([i for i in set(list(dataset['crop']))])
    le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    
    dataset['crop'] = le.transform(dataset['crop'])
    normalized_dataset['crop'] = dataset['crop']
    normalized_dataset['area'] = dataset['area']

    # print(le_name_mapping)
    # pickle.dump(le_name_mapping,open("encodings/encoding_"+current_dataset+".pkl",'wb'))
    return dataset,normalized_dataset
final_dataset,normalized_dataset = normalize_dataset(dataset)

{'Arecanut': 0, 'Arhar/Tur': 1, 'Bajra': 2, 'Banana': 3, 'Black pepper': 4, 'Cotton(lint)': 5, 'Cowpea(Lobia)': 6, 'Dry Ginger': 7, 'Dry chillies': 8, 'Groundnut': 9, 'Horse-gram': 10, 'Jowar': 11, 'Maize': 12, 'Moong(Green Gram)': 13, 'Onion': 14, 'Other  Rabi pulses': 15, 'Other Cereals': 16, 'Other Summer Pulses': 17, 'Peas & beans (Pulses)': 18, 'Potato': 19, 'Ragi': 20, 'Rapeseed &Mustard': 21, 'Rice': 22, 'Sesamum': 23, 'Small millets': 24, 'Sugarcane': 25, 'Sunflower': 26, 'Sweet potato': 27, 'Tapioca': 28, 'Tobacco': 29, 'Turmeric': 30, 'Urad': 31, 'Wheat': 32, 'other oilseeds': 33}


In [ ]:
normalized_dataset

,production,rainfall,n,p,oc,k,crop,area
0,0.000000,0.044226,1.000000,0.654956,0.926335,0.639792,0,67.0
1,0.004244,0.000000,0.992035,0.912220,0.917946,0.175391,0,3149.0
2,0.004337,0.321867,0.992035,0.912220,0.917946,0.175391,0,3541.0
3,0.004329,0.288015,0.992035,0.912220,0.917946,0.175391,0,2836.0
4,0.004795,0.192738,0.992035,0.912220,0.917946,0.175391,0,3514.0
...,...,...,...,...,...,...,...,...
12491,0.151849,0.161616,0.998672,0.964564,0.952936,0.004962,4,88940.0
12492,0.146799,0.378105,0.998672,0.964564,0.952936,0.004962,4,88476.0
12493,0.125302,0.159159,0.998672,0.964564,0.952936,0.004962,4,87132.0
12494,0.146250,0.177177,0.997898,0.932571,0.910784,0.290995,4,91042.0


In [ ]:
# normalized_rain = pd.DataFrame()
# normalized_rain['state'] = final_dataset['state']
# normalized_rain['district'] = final_dataset['district']
# normalized_rain['rainfall'] = normalized_dataset['rainfall']
# normalized_rain.to_excel('normalized_rain_rabi.xlsx',index=False)

In [ ]:
dataset

,crop,state,district,year,season,area,production,yield,rainfall,n,p,oc,k
0,0,Andaman and Nicobar Islands,,2009-10,Summer,4153.0,0.002358,0.50,0.000000,0.9920,0.895564,0.752867,0.8760
1,1,Andaman and Nicobar Islands,,2009-10,Summer,331.0,0.000186,0.50,0.000000,0.9920,0.895564,0.752867,0.8760
2,2,Gujarat,KHEDA,2007-08,Summer,22800.0,0.062678,2.43,0.844037,1.0000,0.166917,0.451900,0.1635
3,2,Gujarat,KHEDA,2006-07,Summer,14400.0,0.040576,2.49,0.690826,1.0000,0.166917,0.451900,0.1635
4,2,Gujarat,KHEDA,2005-06,Summer,14100.0,0.034569,2.16,0.222018,1.0000,0.166917,0.451900,0.1635
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18212,33,Maharashtra,KOLHAPUR,2017-18,Summer,3.0,0.000017,5.00,1.000000,0.2877,0.268048,0.349600,0.2838
18213,33,Maharashtra,NANDURBAR,2016-17,Summer,20.0,0.000010,0.45,1.000000,0.8884,0.066286,0.769900,0.0160
18214,33,Maharashtra,WASHIM,2017-18,Summer,2.0,0.000000,0.20,1.000000,0.5712,0.448583,0.684600,0.0020
18215,33,Maharashtra,KOLHAPUR,2016-17,Summer,160.0,0.000091,0.50,1.000000,0.2877,0.268048,0.349600,0.2838


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
X = np.array(normalized_dataset[['crop','rainfall','n','p','oc','k']])
# y = np.array(normalized_dataset['yield'])
y = np.array(final_dataset['yield'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
reg = LinearRegression()
reg.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
X_train.shape
final_dataset['yield'].describe()

count    18217.000000
mean         3.194949
std          6.647447
min          0.000000
25%          0.750000
50%          1.600000
75%          2.630000
max         63.330000
Name: yield, dtype: float64

In [ ]:
from sklearn.metrics import r2_score
def compare_models(model):
    model_name = model.__class__.__name__
    fit=model.fit(X_train,y_train)
    y_pred=fit.predict(X_test)
    r2=r2_score(y_test,y_pred)
    return([model_name,r2])

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import svm
from sklearn.tree import DecisionTreeRegressor

models = [
    GradientBoostingRegressor(n_estimators=200, max_depth=3, random_state=0),
     RandomForestRegressor(n_estimators=200, max_depth=3, random_state=0),
    svm.SVR(),
   DecisionTreeRegressor()
]

In [ ]:
model_train=list(map(compare_models,models)) 
print(*model_train, sep = "\n")

['GradientBoostingRegressor', 0.916580923626011]
['RandomForestRegressor', 0.8799577093092354]
['SVR', -0.012738960086052353]
['DecisionTreeRegressor', 0.9352570635296462]


In [ ]:
import pickle 
with open('models/GBR'+current_dataset+'.pkl','wb') as f:
    pickle.dump(models[0], f)
with open('models/DTR'+current_dataset+'.pkl','wb') as f:
    pickle.dump(models[-1], f)


In [ ]:
# def get_processed_data():
#     dataset = pd.read_csv('rainfall in india 1901-2015.csv')
#     dataset = dataset.sort_values(by=['YEAR'], ascending=False)
#     output = []
#     states_present = []
#     for index,row in dataset.iterrows():        
#         state = row['SUBDIVISION']
#         if state not in states_present:
#             kharif_rain = (row['JUN']+row['JUL']+row['AUG']+row['SEP']+row['OCT'])/5
#             rabi_rain = (row['NOV']+row['DEC']+row['JAN']+row['FEB']+row['MAR']+row['APR'])/6
#             summer_rain = (row['MAR']+row['APR']+row['MAY']+row['JUN'])/4
#             states_present.append(state)
#             output.append([state,row['YEAR'],kharif_rain,rabi_rain,summer_rain])

#     processed_dataset = pd.DataFrame(np.array(output),columns=['state','year','kharif','rabi','summer'])
#     for col in ['kharif','rabi','summer']:
#         processed_dataset[col] = processed_dataset[col].astype('float')
#     processed_dataset['year'] = processed_dataset['year'].astype('int')
#     processed_dataset['state'] = processed_dataset['state'].astype('string')
#     return processed_dataset
# get_processed_data().to_excel('rainfall-2015.xlsx',index=False)